In [34]:
import numpy as np
import glob
from astropy.io.fits import getdata, getheader, writeto, append, setval
from matplotlib import pyplot as plt


In [1]:

def write_spec(outspec, exp, outname='draft.pha', det_id='det0', mod='A'):
    # Read the template
    null, hdr = getdata('nu30001039002_srcA_sr.pha', 0, header=True)
    writeto(outname, null, header=hdr, overwrite=True)
    spec, hdr = getdata('nu30001039002_srcA_sr.pha', 1, header=True)
    spec['COUNTS'] = outspec
    hdr['RESPFILE'] = '{}{}.rmf'.format(det_id, mod)
    hdr['BACKFILE'] = ''
    hdr['ANCRFILE'] = ''
    hdr['EXPOSURE'] = exp
    append(outname, spec, hdr)



In [67]:
def sort_seqid():
    # Setup NuSTAR time epochs:
    launch_met=77241600. # 2012-06-13T00:00:00
    # Quaterly plots
    
    # Set up data structure:
    seqid_list = {}

        
    for ind, evtdir in enumerate(glob.glob('../scripts/reprocess_background/full_mission/*/')):
        for file in glob.glob('{}/*{}02_cl.evt'.format(evtdir, mod)):
        # Skip these high background obsids
            if file.find("40101012") != -1:
                continue

            if file.find("30161002002") != -1:
                continue

            hdr= getheader(file, 1)
            epoch = np.float(hdr['TSTART'])
            dt_years = (epoch-launch_met) / 3.154e7 # years
            dt_quarter = dt_years
            epoch_ind = np.int(np.floor(dt_quarter))

            e_key = 'epoch{}'.format(epoch_ind)
            if e_key not in seqid_list:
                seqid_list[e_key] = [evtdir]
            else:
                seqid_list[e_key].append(evtdir)
    return seqid_list



def load_data_clean(evtdirs):
    # Returns the full mission data unbinned spectrum
    # set divided into epochs along with exposure.
    
    # Setup NuSTAR time epochs:
    launch_met=77241600. # 2012-06-13T00:00:00
    # Quaterly plots
    
    # Set up data structure:
    data_table = {'A':{}, 'B':{}}
    for mod in data_table:
        for det_id in range(4):
            det_key = 'det{}'.format(det_id)
            data_table[mod][det_key] = {'spec':np.zeros(4096),
                                        'exp':0.}
    
    for ind, evtdir in enumerate(evtdirs):
        for mod in ['A', 'B']:
            for file in glob.glob('{}/*{}02_cl.evt'.format(evtdir, mod)):
            # Skip these high background obsids
                if file.find("40101012") != -1:
                    continue

                if file.find("30161002002") != -1:
                    continue

                hdr= getheader(file, 1)

                evdata = getdata(file, 1)
                
                for det_id in range(4):
                    det_key = 'det{}'.format(det_id)
 
                    evt_filter = ~ (
                                (evdata['STATUS'][:, 8]) | (evdata['STATUS'][:, 7]) \
                                | (evdata['STATUS'][:, 6]) | (evdata['STATUS'][:, 6]) \
                                | (evdata['STATUS'][:, 5]) | (evdata['STATUS'][:,4])
                                ) & (evdata['DET_ID'] == det_id)
#                     this_det = np.where(evdata['DET_ID'] == det_id)[0]
#                     this_evt = evdata[this_det]
#                     evt_filter = ~ (
#                                 (this_evt['STATUS'][:, 8]) | (this_evt['STATUS'][:, 7]) \
#                                 | (this_evt['STATUS'][:, 6]) | (this_evt['STATUS'][:, 6]) \
#                                 | (this_evt['STATUS'][:, 5]) | (this_evt['STATUS'][:,4])
#                                 )

                    good_inds = np.where(evt_filter)[0]
                    ehist, edges = np.histogram(evdata['PI'][good_inds], range = [0, 4096],
                                       bins=4096)            
                    data_table[mod][det_key]['spec'] += ehist
                    data_table[mod][det_key]['exp'] +=np.float(hdr['EXPOSURE'])
        if (ind % 10) ==0:
            print('{} of {}'.format(ind, len(evtdirs)))
        if ind > 10:
            break
    return data_table
    

In [59]:
seqid_list = sort_seqid()

In [69]:
for epoch in seqid_list:
    print(epoch)
    
    evtdirs = seqid_list[epoch]
    dtab = load_data_clean(evtdirs)
    
    for mod in dtab:
        for det_id in dtab[mod]:
            outname='{}_{}_FPM{}_repro.pha'.format(epoch, det_id, mod)
            print(outname)
            write_spec(dtab[mod][det_id]['spec'], 
                       dtab[mod][det_id]['exp'], det_id = det_id, outname=outname)
    break

epoch2
0 of 589
10 of 589
epoch2_det2_FPMA_repro.pha
epoch2_det3_FPMA_repro.pha
epoch2_det0_FPMA_repro.pha
epoch2_det1_FPMA_repro.pha
epoch2_det2_FPMB_repro.pha
epoch2_det3_FPMB_repro.pha
epoch2_det0_FPMB_repro.pha
epoch2_det1_FPMB_repro.pha
